<img src='http://hilpisch.com/taim_logo.png' width="350px" align="right">

# Reinforcement Learning

## OpenAI Gym

Dr Yves J Hilpisch | The AI Machine

http://aimachine.io | http://twitter.com/dyjh

### The use of the "Python 3.10, Tensorflow 2.10" kernel is recommended.

## CartPole

In [ ]:
!git clone https://github.com/tpq-classes/rl_for_finance.git
import sys
sys.path.append('rl_for_finance')


In [ ]:
import warnings; warnings.simplefilter('ignore')

In [ ]:
import gymnasium as gym
import random
import numpy as np
import pandas as pd
from pylab import plt
from IPython import display
plt.style.use('seaborn-v0_8')
np.random.seed(100)

In [ ]:
import tensorflow.compat.v1 as tf
from keras.layers import Dense
from keras.models import Sequential
from keras.optimizers import Adam
from sklearn.metrics import accuracy_score
tf.random.set_random_seed(100)
tf.logging.set_verbosity(tf.logging.ERROR)

In [ ]:
from tensorflow.python.framework.ops import disable_eager_execution
disable_eager_execution()

In [ ]:
env = gym.make('CartPole-v1')

## Setting Seeds

In [ ]:
def set_seeds(seed=100):
    random.seed(seed)
    np.random.seed(seed)
    # env.seed(seed)

## Neural Network Agent

In [ ]:
class NNAgent:
    def __init__(self):
        self.max = 0
        self.scores = list()
        self.memory = list()
        self.epsilon = 0.5
        self.model = self._build_model()
        
    def _build_model(self):
        model = Sequential()
        model.add(Dense(1024, input_dim=4,
                        activation='relu'))
        model.add(Dense(1, activation='sigmoid'))
        model.compile(loss='binary_crossentropy',
                      optimizer=Adam(lr=0.001))
        return model
        
    def act(self, state):
        if random.random() <= self.epsilon:
            return env.action_space.sample()  # exploration
        action = np.where(self.model.predict(
            state, batch_size=None, verbose=False)[0, 0] > 0.5, 1, 0)  # exploitation
        return action
                    
    def train_model(self, state, action):
        self.model.fit(state, np.array([action,]),
                       epochs=1, verbose=False)
    
    def learn(self, episodes):
        for e in range(1, episodes + 1):
            state, info = env.reset()
            for _ in range(201):
                state = np.reshape(state, [1, 4])
                action = self.act(state)
                next_state, reward, done, trunc, info = env.step(action)
                if done:
                    score = _ + 1
                    self.scores.append(score)
                    self.max = max(score, self.max)
                    print('episode: {:4d}/{} | score: {:3d} | max: {:3d}'
                          .format(e, episodes, score, self.max), end='\r')
                    break
                self.train_model(state, action)
                self.memory.append((state, action))
                state = next_state

In [ ]:
set_seeds(100)
agent = NNAgent()

In [ ]:
episodes = 10000

In [ ]:
%time agent.learn(episodes)

In [ ]:
sum(agent.scores) / len(agent.scores)  # average score

## Accuracy Score

In [ ]:
f = np.array([m[0][0] for m in agent.memory])  # features
f

In [ ]:
l = np.array([m[1] for m in agent.memory])  # labels
l

In [ ]:
accuracy_score(np.where(agent.model.predict(f) > 0.5, 1, 0), l)  # prediction accuracy

<img src='http://hilpisch.com/taim_logo.png' width="350px" align="right">

<br><br><br><a href="http://tpq.io" target="_blank">http://tpq.io</a> | <a href="http://twitter.com/dyjh" target="_blank">@dyjh</a> | <a href="mailto:ai@tpq.io">ai@tpq.io</a>